In [1]:
#Dependencies and Setup
!pip install psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import csv
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import NVARCHAR
from sqlalchemy import INT

In [2]:
#Load the files 
file_loaded = "./tabular_aadt_2015.csv"
file_loaded_next = "./tabular_aadt_2016.csv"
file_loaded_next_next = "./tabular_aadt_2017.csv"

In [3]:
#Read CSV file containing 2015 DATA & populate a new dataframe for it
df = pd.read_csv(file_loaded)
df

,District,Route,County,Postmile,Description,Back_Peak_Hour,Back_Peak_Month,Back_AADT,Ahead_Peak_Hour,Ahead_Peak_Month,Ahead_AADT
0,12,1,ORA,R0.129,"DANA POINT, JCT. RTE. 5",0,0,0,3800,41000,37750
1,12,1,ORA,R.78,"DANA POINT, DOHENY PARK ROAD",3800,41000,37750,3900,42000,38800
2,12,1,ORA,4.864,BREAK IN ROUTE,0,0,0,0,0,0
3,12,1,ORA,8.43,"LAGUNA BEACH, MOUNTAIN ROAD",2900,39500,36800,2900,39500,36800
4,12,1,ORA,9.418,"LAGUNA BEACH, JCT. RTE. 133 NORTH",3050,41000,38500,3450,41000,38500
5,12,1,ORA,9.6,"LAGUNA BEACH, CLIFF DRIVE/ ASTER STREET",3400,40000,37400,3400,40000,37400
6,12,1,ORA,11.5,"LAGUNA BEACH, NORTH CITY LIMITS",3200,38000,35700,3200,39000,35700
7,12,1,ORA,14.057,BREAK IN ROUTE,0,0,0,0,0,0
8,12,1,ORA,R18.073,"NEWPORT BEACH, BAYSIDE DRIVE",4050,50000,45850,4850,60000,54800
9,12,1,ORA,R18.446,"NEWPORT BEACH, DOVER DRIVE",4300,53000,48650,4300,53000,48650


In [4]:
#Extract the columns from the 2015 CSV file ('District', 'County','Postmile') and Poplulate a new DataFrame
df_columns = ["District","County","Postmile"]
df_transformed_2015 = df[df_columns]
df_transformed_2015['Year']= 2015
df_transformed_2015.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,District,County,Postmile,Year
0,12,ORA,R0.129,2015
1,12,ORA,R.78,2015
2,12,ORA,4.864,2015
3,12,ORA,8.43,2015
4,12,ORA,9.418,2015


In [5]:
#Read CSV file containing 2016 DATA & populate a new dataframe for it
df_next = pd.read_csv(file_loaded_next)
df_next.head()

,District,Route,County,Postmile,Description,Back_Peak_Hour,Back_Peak_Month,Back_AADT,Ahead_Peak_Hour,Ahead_Peak_Month,Ahead_AADT
0,12,1,ORA,R.129,"DANA POINT, JCT. RTE. 5",0,0,0,3800,41000,37750
1,12,1,ORA,R.78,"DANA POINT, DOHENY PARK ROAD",3800,41000,37750,3900,42000,38800
2,12,1,ORA,R1.077,BREAK IN ROUTE,0,0,0,0,0,0
3,12,1,ORA,8.43,"LAGUNA BEACH, MOUNTAIN ROAD",2900,39500,36800,2900,39500,36800
4,12,1,ORA,9.418,"LAGUNA BEACH, JCT. RTE. 133 NORTH",3050,41000,38500,3450,41000,38500


In [6]:
#Extract the columns from the 2016 CSV file ('District', 'County','Postmile') and Poplulate a new DataFrame
df_columns_new = ["District","County","Postmile"]
df_transformed_2016 = df[df_columns_new]
df_transformed_2016['Year']= 2016
df_transformed_2016.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,District,County,Postmile,Year
0,12,ORA,R0.129,2016
1,12,ORA,R.78,2016
2,12,ORA,4.864,2016
3,12,ORA,8.43,2016
4,12,ORA,9.418,2016


In [7]:
#Read CSV file containing 2017 DATA & populate a new dataframe for it
df_next_next = pd.read_csv(file_loaded_next_next)
df_next_next.head()

,District,Route,County,Postmile,Description,Back_Peak_Hour,Back_Peak_Month,Back_AADT,Ahead_Peak_Hour,Ahead_Peak_Month,Ahead_AADT
0,12,1,ORA,R0.129,"DANA POINT, JCT. RTE. 5",0,0,0,3800,41000,37750
1,12,1,ORA,R0.78,"DANA POINT, DOHENY PARK ROAD",3800,41000,37750,3900,42000,38800
2,12,1,ORA,R1.077,BREAK IN ROUTE,0,0,0,0,0,0
3,12,1,ORA,8.43,"LAGUNA BEACH, MOUNTAIN ROAD",2900,39500,36800,2900,39500,36800
4,12,1,ORA,9.418,"LAGUNA BEACH, JCT. RTE. 133 NORTH",3050,41000,38500,3450,41000,38500


In [8]:
#Extract the columns from the 2017 CSV file ('District', 'County','Postmile') and Poplulate a new DataFrame
df_columns_new_new = ["District","County","Postmile"]
df_transformed_2017 = df[df_columns_new_new]
df_transformed_2017['Year']= 2017
df_transformed_2017.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,District,County,Postmile,Year
0,12,ORA,R0.129,2017
1,12,ORA,R.78,2017
2,12,ORA,4.864,2017
3,12,ORA,8.43,2017
4,12,ORA,9.418,2017


In [10]:
#Combine all three new DataFrames together into one big DataFrame WITHOUT A MERGE 
dfs = [df_transformed_2017, df_transformed_2016, df_transformed_2015]
dfs

[      District County  Postmile  Year
 0           12    ORA    R0.129  2017
 1           12    ORA      R.78  2017
 2           12    ORA     4.864  2017
 3           12    ORA      8.43  2017
 4           12    ORA     9.418  2017
 5           12    ORA       9.6  2017
 6           12    ORA      11.5  2017
 7           12    ORA    14.057  2017
 8           12    ORA   R18.073  2017
 9           12    ORA   R18.446  2017
 10          12    ORA    19.797  2017
 11          12    ORA     20.37  2017
 12          12    ORA    21.549  2017
 13          12    ORA     22.09  2017
 14          12    ORA    23.739  2017
 15          12    ORA     25.89  2017
 16          12    ORA     29.89  2017
 17          12    ORA    32.721  2017
 18          12    ORA    33.719  2017
 19           7     LA         0  2017
 20           7     LA      1.86  2017
 21           7     LA     1.973  2017
 22           7     LA     2.751  2017
 23           7     LA     5.011  2017
 24           7     LA   

In [11]:
#Combine 2015 & 2016 new DataFrames together into one big DataFrame WITH A MERGE
final_df = pd.merge(df_transformed_2015,df_transformed_2016, on = 'Year', how ='outer')
final_df.head()

,District_x,County_x,Postmile_x,Year,District_y,County_y,Postmile_y
0,12.0,ORA,R0.129,2015,NaN,NaN,NaN
1,12.0,ORA,R.78,2015,NaN,NaN,NaN
2,12.0,ORA,4.864,2015,NaN,NaN,NaN
3,12.0,ORA,8.43,2015,NaN,NaN,NaN
4,12.0,ORA,9.418,2015,NaN,NaN,NaN


In [12]:
#Merge 2017 DataFrame with the merged 2015 & 2016 Dataframe
merged_df = pd.merge(final_df, df_transformed_2017, on='Year', how='outer')
merged_df.head()

,District_x,County_x,Postmile_x,Year,District_y,County_y,Postmile_y,District,County,Postmile
0,12.0,ORA,R0.129,2015,NaN,NaN,NaN,NaN,NaN,NaN
1,12.0,ORA,R.78,2015,NaN,NaN,NaN,NaN,NaN,NaN
2,12.0,ORA,4.864,2015,NaN,NaN,NaN,NaN,NaN,NaN
3,12.0,ORA,8.43,2015,NaN,NaN,NaN,NaN,NaN,NaN
4,12.0,ORA,9.418,2015,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Concat the three DataFrames into one big DataFrame
frames  = [df_transformed_2015,df_transformed_2016,df_transformed_2017]
final_df = pd.concat(frames)
final_df.head()

,District,County,Postmile,Year
0,12,ORA,R0.129,2015
1,12,ORA,R.78,2015
2,12,ORA,4.864,2015
3,12,ORA,8.43,2015
4,12,ORA,9.418,2015


In [14]:
#Create an Engine that connects Pandas to pgAdmine
Base = automap_base()
connection_string = "postgres:ghaithramahi1@localhost:5433/California_Traffic"
engine = create_engine(f'postgresql://{connection_string}')

#Engine Connection
conn = engine.raw_connection()

In [15]:
#Confirm tables
engine.table_names()

['test', 'CA_Traffic', 'premise', 'betyouthisworks', 'hi', 'ca_traffic']

In [16]:
#Print and confirm final DataFrame
# final_df['County'] = str(final_df['County'])
# final_df['Postmile'] = str(final_df['Postmile'])
# final_df.groupby('Postmile').count()
final_df.dtypes
#final_df.columns = ['district','county','postmile','year']

District     int64
County      object
Postmile    object
Year         int64
dtype: object

In [17]:
final_df.head()

,District,County,Postmile,Year
0,12,ORA,R0.129,2015
1,12,ORA,R.78,2015
2,12,ORA,4.864,2015
3,12,ORA,8.43,2015
4,12,ORA,9.418,2015


In [18]:
#Load DataFrames into pgAdmin
final_df.to_sql(name='California_Traffic', con=engine, if_exists='append', index=False, schema = 'public')

In [ ]:
final_df.head(15)

In [ ]:
#Extra software version check
!pip freeze

In [ ]:
#Reconfirm Database
len(final_df['County'][0])